In [2]:
import tensorflow as tf
import glob
import os
import pandas as pd

In [3]:
import tensorboard

In [5]:
import tensorboard.data.experimental

In [6]:
import tensorboard.data

In [7]:
import tensorflow.python

In [10]:
import tensorflow.python.summary

In [175]:
opts.normalize

False

In [22]:
import os
import sys

In [29]:
logfiles = glob.glob('/home/ben/Documents/research/repbrep/training_logs/**/*event*', recursive=True)

In [30]:
si = tf.compat.v1.train.summary_iterator(logfiles[0])

In [9]:
from tqdm.notebook import tqdm
import tensorflow as tf
import glob
import os
import pandas as pd
logfiles = glob.glob('/home/ben/Documents/research/repbrep/training_logs/**/*event*', recursive=True)
records = []
runlog = pd.DataFrame(columns=['logfile', 'wall_time', 'step', 'metric','value'])
for logfile in tqdm(logfiles):
    si = tf.compat.v1.train.summary_iterator(logfile)
    for e in si:
        wall_time = e.wall_time
        step = e.step
        for v in e.summary.value:
            r = {'logfile':logfile, 'wall_time':wall_time, 'step':step, 'metric':v.tag, 'value':v.simple_value}
            records.append(r)
tf_df = pd.DataFrame.from_records(records)

  0%|          | 0/1003 [00:00<?, ?it/s]

In [12]:
tf_df.to_parquet('/home/ben/Documents/research/repbrep/results/tf_logs_summary.parquet')

In [26]:
tf_df.logfile.iloc[0].split('/')

['',
 'home',
 'ben',
 'Documents',
 'research',
 'repbrep',
 'training_logs',
 'uvnet-fusiongallery-fixed-steps',
 'fusiongallery-fixed-steps',
 'simple_train_test-1000-4',
 '0504.214924',
 'events.out.tfevents.1651726175.sanka.cs.washington.edu.2728977.0']

In [59]:
def parse_logfile_name(x):

    full_mfcad_size = 13940
    full_f360seg_size = 23266
    full_fabwave_size = 2613

    name = x['logfile'].lower()
    parts = name.split('/')
    base_dir = parts[7]
    if base_dir == 'ours':
        model = 'ours'
        dataset = parts[8].split('_')[0]
        exp = parts[9].split('_')
        train_size = -1
        seed = -1
        if exp[0] == 'mp2' or exp[0] == 'fwp':
            train_size = float(exp[1])
            if train_size <= 1.0:
                train_size = full_fabwave_size * train_size
            train_size = int(train_size)
            seed = int(exp[2])
    else:
        bd_p = base_dir.split('-')
        model = bd_p[0]
        dataset = bd_p[1]
        if dataset == 'fusiongallery':
            dataset = 'f360seg'
        train_size = -1
        seed = -1
        for p in parts:
            pp = p.split('-')
            if len(pp) > 2:
                if pp[-1] in [str(x) for x in range(10)]:
                    seed = int(pp[-1])
                    if pp[-2] == 'full':
                        if dataset == 'mfcad':
                            train_size = full_mfcad_size
                        elif dataset == 'fabwave':
                            train_size = full_fabwave_size
                        else:
                            train_size = full_f360seg_size
                    else:
                        train_size = int(pp[-2])
                        if dataset == 'fabwave':
                            ts_float = float(train_size)/100
                            train_size = int(ts_float * full_fabwave_size)
        exp = parts[-3]

    return pd.Series([dataset, model, train_size, seed])
    
newcols = tf_df.apply(parse_logfile_name, axis=1)
newcols.columns = ['dataset','model','train_size','seed']
newdf = tf_df.join(newcols)

In [72]:
newdf[newdf.seed >= 0].to_parquet('/home/ben/Documents/research/repbrep/results/tf_events.parquet')

In [73]:
events_df = newdf[newdf.seed >= 0]

In [99]:
events_df_merged = pd.merge(events_df, start_times, on=['logfile','metric'])

In [101]:
events_df_merged['rel_time'] = events_df_merged.wall_time - events_df_merged.start_time

In [103]:
events_df_merged.to_parquet('/home/ben/Documents/research/repbrep/results/tf_events.parquet')

In [85]:
start_times = events_df[['logfile','metric','wall_time']].groupby(['logfile','metric']).agg(min).reset_index().rename(columns={'wall_time':'start_time'})

In [88]:
events_df['joinkey'] = events_df.logfile + events_df.metric

/tmp/ipykernel_61840/2001750339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['joinkey'] = events_df.logfile + events_df.metric


In [89]:
start_times['joinkey'] = start_times.logfile + start_times.metric

In [143]:
def test_function(x):    
    min_time = x.rel_time.values[x.value.values.argmin()]
    max_time = x.rel_time.values[x.value.values.argmax()]
    return pd.Series([min_time, max_time], index=['min_time','max_time'])

agged = edf.groupby(['dataset','model','train_size','seed','metric']).apply(test_function).reset_index()#.groupby(['dataset','model','train_size','metric']).agg('mean').reset_index()

In [141]:
import altair as alt

In [164]:
agged.groupby(['dataset','model','train_size','metric']).agg({'min_time':['mean', 'std']})

min_time  \
                                                                mean   
dataset model   train_size metric                                      
f360seg brepnet 10         _hparams_/experiment             0.000000   
                           _hparams_/session_end_info       0.000000   
                           _hparams_/session_start_info     0.000000   
                           epoch                            0.000000   
                           hp_metric                        0.000000   
...                                                              ...   
mfcad   uvnet   13940      train_iou                        0.000000   
                           train_loss                    1535.750187   
                           val_accuracy                     0.000000   
                           val_iou                          0.000000   
                           val_loss                      1291.886662   

                                                                     
                                                                std  
dataset model   train_size metric                                    
f360seg brepnet 10         _hparams_/experiment            0.000000  
                           _hparams_/session_end_info      0.000000  
                           _hparams_/session_start_info    0.000000  
                           epoch                           0.000000  
                           hp_metric                       0.000000  
...                                                             ...  
mfcad   uvnet   13940      train_iou                       0.000000  
                           train_loss                    444.662759  
                           val_accuracy                    0.000000  
                           val_iou                         0.000000  
                           val_loss                      554.990425  

[790 rows x 2 columns]

In [157]:
agged2 = agged.groupby(['dataset','model','train_size','metric']).agg(['mean', 'stdev']).reset_index()
alt.Chart(agged2[(agged2.metric.str.contains('val')) & (agged2.metric.str.contains('loss')) & (agged2.dataset == 'mfcad')])\
    .mark_circle(size=60)\
    .encode(
        x=alt.X('train_size', scale=alt.Scale(type='log')),
        y=alt.Y('min_time', scale=alt.Scale(type='log')),
        color='model'
    )

AttributeError: 'SeriesGroupBy' object has no attribute 'stdev'

In [124]:
edf[(edf.model == 'ours') & (edf.metric == 'val_loss_epoch') & (edf.seed == 0) & (edf.dataset == 'f360seg') & (edf.train_size == 100)]

,logfile,wall_time,step,metric,value,dataset,model,train_size,seed,start_time,rel_time
1010373,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,0,val_loss_epoch,2.658961,f360seg,ours,100,0,1.652724e+09,0.000000
1010374,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,1,val_loss_epoch,2.463522,f360seg,ours,100,0,1.652724e+09,0.260975
1010375,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,2,val_loss_epoch,2.291942,f360seg,ours,100,0,1.652724e+09,0.475332
1010376,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,3,val_loss_epoch,2.144808,f360seg,ours,100,0,1.652724e+09,0.644731
1010377,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,4,val_loss_epoch,2.025117,f360seg,ours,100,0,1.652724e+09,0.978386
...,...,...,...,...,...,...,...,...,...,...,...
1010556,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,183,val_loss_epoch,2.213193,f360seg,ours,100,0,1.652724e+09,59.055521
1010557,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,184,val_loss_epoch,2.225710,f360seg,ours,100,0,1.652724e+09,59.718734
1010558,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,185,val_loss_epoch,2.238228,f360seg,ours,100,0,1.652724e+09,60.049240
1010559,/home/ben/Documents/research/repbrep/training_...,1.652724e+09,186,val_loss_epoch,2.251573,f360seg,ours,100,0,1.652724e+09,60.349135


In [116]:
edf[(edf.model != 'ours')].metric.unique()

array(['hp_metric', '_hparams_/experiment',
       '_hparams_/session_start_info', '_hparams_/session_end_info',
       'train_loss', 'train_iou', 'train_accuracy', 'epoch', 'val_loss',
       'val_iou', 'val_accuracy', 'global_step', 'test/accuracy',
       'test/mean_iou', 'test/ExtrudeSide_iou', 'test/ExtrudeEnd_iou',
       'test/CutSide_iou', 'test/CutEnd_iou', 'test/Fillet_iou',
       'test/Chamfer_iou', 'test/RevolveSide_iou', 'test/RevolveEnd_iou',
       'validation/loss', 'validation/accuracy', 'validation/mean_iou',
       'validation/ExtrudeSide_iou', 'validation/ExtrudeEnd_iou',
       'validation/CutSide_iou', 'validation/CutEnd_iou',
       'validation/Fillet_iou', 'validation/Chamfer_iou',
       'validation/RevolveSide_iou', 'validation/RevolveEnd_iou',
       'train/loss', 'train/accuracy', 'loss', 'train_acc', 'val_acc',
       'validation/rectangular_through_slot_iou',
       'validation/triangular_through_slot_iou',
       'validation/rectangular_passage_iou',
   

In [110]:
edf.model.value_counts()

ours       976670
uvnet      259720
brepnet     92434
Name: model, dtype: int64